### Import libraries

pandas - data analysis and manipulation

numpy - support for multidimensional arrays and matrices

genomepy -  handling genomes and gene annotations

matplotlib - creating visualisations 

SciPy - scientific and technical computing

In [ ]:
pip install genomepy

In [ ]:
import pandas as pd
import numpy as np
import genomepy
import matplotlib.pyplot as plt
from scipy import stats
import random

### Download genome assembly with genomepy and read in the gene annotation

In [ ]:
genomepy.install_genome("GRCm39", "NCBI", annotation = False)

gencode = pd.read_table("gencode.vM32.annotation.gff3.gz", comment="#", sep = "\t", names = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute'])
gencode.head() 

### Select genes only

In [ ]:
gencode_genes = gencode[(gencode.feature == "gene")][['seqname', 'source', 'feature','start', 'end', 'strand', 'attribute']].copy().reset_index().drop('index', axis=1)
gencode_genes.head()

### Extract gene names and gene types

In [ ]:
def gene_info(x):
    g_name = list(filter(lambda x: 'gene_name' in x,  x.split(";")))[0].split("=")[1]
    g_type = list(filter(lambda x: 'gene_type' in x,  x.split(";")))[0].split("=")[1]
    return (g_name, g_type)
gencode_genes["gene_name"], gencode_genes["gene_type"] = zip(*gencode_genes.attribute.apply(lambda x: gene_info(x)))

### Select protein-coding genes only

In [ ]:
gencode_genes = gencode_genes[gencode_genes['gene_type'] == 'protein_coding'].reset_index().drop('index', axis=1)
gencode_genes.head()

### Index by gene name

In [ ]:
gencode_genes = gencode_genes.set_index('gene_name')
gencode_genes.head()

### Create dictionary of genes with their esitmated elongation rates 

In [ ]:
df = pd.read_excel('gene_list.xlsx') # A list of genes for which transcription elongation rates have been estimated in mESCs by Jonkers et al. in the times spanning  12.5-25 minutes following flavopiridol treatment 

genedata = dict(zip(df['Gene name'].tolist(), df['Rate (bp/min)'].tolist()))

print(genedata)

### Determine gene coordinates

In [ ]:
gene_coordinates = pd.DataFrame({'gene_name': [], 'chromosome': [], 'start': [], 'end': [], 'strand': []})
for gene in genedata:
    try:
        a = gencode_genes.loc[gene]['seqname']
        b = gencode_genes.loc[gene]['start']
        c = gencode_genes.loc[gene]['end']
        d = gencode_genes.loc[gene]['strand']
        gene_coordinates.loc[len(gene_coordinates)] = [gene, a, b, c, d]
    except Exception as e:
        pass
gene_coordinates = gene_coordinates.set_index('gene_name')
gene_coordinates.head()

### Select exons

In [ ]:
gencode_exons = gencode[(gencode.feature == "exon")][['seqname', 'source', 'feature','start', 'end', 'strand', 'attribute']].copy().reset_index().drop('index', axis=1)

gencode_exons["gene_name"], gencode_exons["gene_type"] = zip(*gencode_exons.attribute.apply(lambda x: gene_info(x)))

gencode_exons = gencode_exons[gencode_exons['gene_type'] == 'protein_coding'].reset_index().drop('index', axis=1)

gencode_exons = gencode_exons.set_index('gene_name')

gencode_exons1 = gencode_exons.set_index('attribute')

print(gencode_exons1)

### Determine exon coordinates 

In [ ]:
exon_counts = pd.DataFrame({'gene_name': [], 'exon_count': [], 'exons': [], 'exons_coordinates': [], 'Rate (bp/min)':[]})
for gene in genedata:
    exon_count = 0
    exons = []
    exon_coordinates = []
    try: 
        a = gene_coordinates.loc[gene]['chromosome']
        b = gene_coordinates.loc[gene]['start']
        c = gene_coordinates.loc[gene]['end']
        d = gene_coordinates.loc[gene]['strand']
        for attribute in gencode_exons.loc[gene]['attribute']:
            if "Ensembl" in attribute:
                start = gencode_exons1.loc[attribute]['start']
                end = gencode_exons1.loc[attribute]['end']
                if d == '+':
                    exon_count = exon_count + 1
                    exon_length = end-start
                    exons.append(exon_length)
                    exon_coordinates.append((a, start, end))
                if d == '-':
                    exon_count = exon_count + 1
                    exon_length = end-start
                    exons.append(exon_length)
                    exon_coordinates.append((a, start, end))
    except:
        pass
    if exon_count >-1:
        exon_counts.loc[len(exon_counts)] = [gene, exon_count, exons, exon_coordinates, genedata[gene]]

exon_counts = exon_counts.set_index('gene_name')

### Define a function to highlight a given sequence

In [ ]:
def highlight_sequence(sequence, larger_sequence):
    highlighted_sequence = larger_sequence.replace(sequence, sequence.upper())
    return highlighted_sequence

## Modified model incorporating exon sequences

In [ ]:
# The model simulates the Pol II movement in time slices of 1/30 seconds

# Define the parameters
initiation_rate = 1/(60 * 30)  # 1 event per 1 minute
elongation_rate = 1800.0 / (60.0 *30)  # 1800 bp per minute 
termination_rate = 1/(60 * 30)  # 1 event per 1 minute 
pause_escape_rate = 1/ (60 * 30)  # 1 event per minute
simulation_time = 6*3600 *30  # 6 hours simulation 
footprint = 34 # Pol II footprint
num_simulations = 100 #number of simulations

#Define the exon factor
exon_factor = 3 
intron_factor = 1 
base_factor = {'A': 1/(exon_factor), 'T': 1/(exon_factor), 'C': 1/(exon_factor), 'G': 1/(exon_factor), 'a': 1/(intron_factor), 't': 1/(intron_factor), 'c': 1/(intron_factor), 'g': 1/(intron_factor)} 

#Specify the gene
gene = 'Wapl'
a = gencode_genes.loc[gene]['seqname']
b = gencode_genes.loc[gene]['start']
c = gencode_genes.loc[gene]['end']
d = gene_coordinates.loc[gene]['strand']
x = genomepy.Genome("GRCm39").get_seq(a[3:],b,c)
x = (str(x)).lower()
for exon in exon_counts.loc[gene]['exons_coordinates']:
    e = exon[1]
    f = exon[2]
    g = (str((genomepy.Genome("GRCm39").get_seq(a[3:],e,f))).lower())
    x = highlight_sequence(g, x)
gene = x 

gene_length = len(gene) 

# Initialize the list of Pol II positions for all simulations
final_positions = []

# Simulate the Pol II movement for multiple simulations
for sim in range(num_simulations):
    # Initialize the Pol II positions
    pol_positions = []

    # Simulate the Pol II movement
    for time in range(simulation_time):
        
        # INITIATION
        # Check if a new Pol II should initiate
        if random.random() < initiation_rate:
            if len([p for p in pol_positions if p < footprint]) == 0:
                pol_positions.append(0)
        
        # PAUSE ESCAPE
        # Resume paused Pol II
        if 0 in pol_positions:
            if random.random() < pause_escape_rate:
                    pol_positions.remove(0)
                    pol_positions.append(1)
                    
        # ELONGATION
        # Sort the Pol II positions in ascending order
        pol_positions.sort()

        # Adjust the effective elongation rate based on the number of Pol II molecules (since only one Pol II molecule is randomly selected to move, the average elongation rate needs to be adjusted)
        num_pol = len(pol_positions)
        elongation_rate_effective = elongation_rate * num_pol

        # Randomly select an Pol II molecule to move
        if pol_positions:
            i = random.randint(0, len(pol_positions)-1)

            try:
                base = gene[int(pol_positions[i])]
            except:
                pass
                
            # If the Pol II is paused, it cannot move until it resumes
            if pol_positions[i] == 0:
                continue

            # If the Pol II is not paused
            else:
                # If the Pol II is the last one or is not blocked by the previous Pol II
                if i == len(pol_positions)-1:
                    if pol_positions[i] < gene_length:
                        pol_positions[i] += elongation_rate_effective * base_factor[base]
                    else:
                        continue
                elif pol_positions[i] + elongation_rate_effective + footprint < pol_positions[i+1]:
                    if pol_positions[i] < gene_length:
                        pol_positions[i] += elongation_rate_effective * base_factor[base]
                    else:
                        continue
                        
        # TERMINATION
        # Check if any Pol II molecules has reached the end of the gene
        for p in pol_positions:
            if p >= gene_length:
                if random.random() < termination_rate:
                    pol_positions.remove(p)
                else:
                    continue
            else:
                continue

    # Record the final positions of the Pol II molecules
    final_positions.append(pol_positions)
    
# Calculate the average position of each Pol II
avg_positions = np.mean(final_positions, axis=0)*num_simulations

# Bin the Pol II positions into 100 bins
hist, bins = np.histogram(avg_positions, bins=(int(gene_length/1000)+1))

# Calculate the mean height, standard deviation, and coefficient of variation of the 100 bins
mean_height = np.mean(hist)
std_dev = np.std(hist)
coeff_of_var = std_dev/mean_height

# Print the gene signature
print("Gene signature:")
print(f"Mean height: {mean_height}")
print(f"Standard deviation: {std_dev}")
print(f"Coefficient of variation: {coeff_of_var}")

# Visualize the distribution of Pol II positions
plt.hist(avg_positions, bins=(int(gene_length/1000)+1))
plt.xlabel('Position on the gene (bp)', fontsize = 15)
plt.ylabel('Pol II count', fontsize = 15)
plt.ylim(0, 200)
n = len(avg_positions)
plt.text(0.70, 0.9, f'n = {n}', transform=plt.gca().transAxes, fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()